# モデルをトレーニングする

機械学習の中心的な目的は、アプリケーションで使用できる予測モデルをトレーニングすることです。Azure Machine Learning では、スクリプトを使用して、Scikit-Learn、TensorFlow、PyTorch、SparkML などの一般的な機械学習フレームワークを活用してモデルをトレーニングできます。これらのトレーニング スクリプトを実験として実行して、メトリックと出力、特にトレーニング済みのモデルを追跡できます。

## ワークスペースに接続する

まず、Azure ML SDK を使用してワークスペースに接続する必要があります。

> **注**: 前回の演習を完了してから Azure サブスクリプションとの認証済みセッションの有効期限が切れている場合は、再認証を求めるメッセージが表示されます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 保存した構成ファイルからワークスペースを読み込む
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## トレーニング スクリプトを作成する

Python スクリプトを使用して、糖尿病データに基づいて機械学習モデルをトレーニングします。スクリプトとデータ ファイル用のフォルダーを作成することから始めましょう。

In [ ]:
import os, shutil

# 実験ファイル用フォルダーを作成する
training_folder = 'diabetes-training'
os.makedirs(training_folder, exist_ok=True)

# データ ファイルを実験フォルダーにコピーする
shutil.copy('data/diabetes.csv', os.path.join(training_folder, "diabetes.csv"))

これで、トレーニング スクリプトを作成してフォルダーに保存する準備が整いました。

In [ ]:
%%writefile $training_folder/diabetes_training.py
# ライブラリをインポートする
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# 実験実行コンテキストを取得する
run = Run.get_context()

# 糖尿病データセットを読み込む
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# フィーチャーとラベルを分離する
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# データをトレーニング セットとテスト セットに分割する
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 正規化ハイパーパラメーターを設定する
reg = 0.01

# ロジスティック回帰モデルをトレーニングする
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# 精度を計算する
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC を計算する
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# トレーニング済モデルを出力フォルダーに保存する。
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

## Estimator を使用してスクリプトを実験として実行する

以前は、**実行構成**と**スクリプト実行構成**を使用して実験スクリプトを実行しました。同じ方法でトレーニング スクリプトを実行できますが、これらの構成は両方とも 1 つのオブジェクトに抽象化されるので、通常 **Estimator** を使用する方が簡単です。

この場合、汎用 **Estimator** オブジェクトを使用して、トレーニング実験を実行します。この Estimator の既定環境には、**Scikit-learn** パッケージが含まれていないので、これを構成に明示的に追加する必要があります。Conda 環境は、初めて Estimator を使用するときにオンデマンドで構築され、同じ構成を使用する将来の実行用にキャッシュされます。したがって、初めての実行は少し時間がかかります。

In [ ]:
from azureml.train.estimator import Estimator
from azureml.core import Experiment

# Estimator を作成する
estimator = Estimator(source_directory=training_folder,
                      entry_script='diabetes_training.py',
                      compute_target='local',
                      conda_packages=['scikit-learn']
                      )

# 実験を作成する
experiment_name = 'diabetes-training'
experiment = Experiment(workspace = ws, name = experiment_name)

# Estimator に基づいて実験を実行する
run = experiment.submit(config=estimator)
run.wait_for_completion(show_output=True)

実験の実行と同様に、**RunDetails** ウィジェットを使用して実行に関する情報を表示し、Azure Machine Learning Studio での実行に対するリンクを取得できます。

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

**Run** オブジェクトからメトリックと出力を取得することもできます。

In [ ]:
# 指標の記録を取得する
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

## トレーニング済みモデルを登録する

実験の出力には、トレーニング済みモデル ファイル (**diabetes_model.pkl**) が含まれることに注意してください。このモデルを Azure Machine Learning ワークスペースに登録すると、モデルのバージョンを追跡し、後で取得できるようになります。

In [ ]:
from azureml.core import Model

# モデルを登録する
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Estimator'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# 登録済みモデルを一覧表示する
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

## パラメーター化されたトレーニング スクリプトを作成する

スクリプトにパラメーターを追加することで、トレーニング実験の柔軟性を高めることができ、同じトレーニング実験を異なる設定で繰り返すことができます。この場合、モデルのトレーニング時にロジスティック回帰アルゴリズムで使用される正規化率のパラメーターを追加します。

パラメーター化されたスクリプトとトレーニング データ用のフォルダーをもう一度作成します。

In [ ]:
import os, shutil

# 実験ファイル用フォルダーを作成する
training_folder = 'diabetes-training-params'
os.makedirs(training_folder, exist_ok=True)

# データ ファイルを実験フォルダーにコピーする
shutil.copy('data/diabetes.csv', os.path.join(training_folder, "diabetes.csv"))

次に、正規化率ハイパーパラメーターのパラメーターを含むスクリプトを作成します。

In [ ]:
%%writefile $training_folder/diabetes_training.py
# ライブラリをインポートする
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
import argparse
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# 実験実行コンテキストを取得する
run = Run.get_context()

# 正規化ハイパーパラメーターを設定する
parser = argparse.ArgumentParser()
parser.add_argument('--reg_rate', type=float, dest='reg', default=0.01)
args = parser.parse_args()
reg = args.reg

# 糖尿病データセットを読み込む
print("Loading Data...")
# 糖尿病データセットを読み込む
diabetes = pd.read_csv('diabetes.csv')

# フィーチャーとラベルを分離する
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# データをトレーニング セットとテスト セットに分割する
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# ロジスティック回帰モデルをトレーニングする
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# 精度を計算する
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC を計算する
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

## フレームワークに固有の Estimator を使用する

汎用 **Estimator** クラスを使用してトレーニング スクリプトを実行しましたが、一般的な機械学習フレームワークの環境定義を含むフレームワーク固有の Estimator を利用することもできます。この場合は、Scikit-Learn を使用しているので**SKLearn** Estimator を使用できます。つまり、構成で **Scikit-learn** パッケージを指定する必要はありません。

In [ ]:
from azureml.train.sklearn import SKLearn
from azureml.widgets import RunDetails

# Estimator を作成する
estimator = SKLearn(source_directory=training_folder,
                    entry_script='diabetes_training.py',
                    script_params = {'--reg_rate': 0.1},
                    compute_target='local'
                    )

# 実験を作成する
experiment_name = 'diabetes-training'
experiment = Experiment(workspace = ws, name = experiment_name)

# 実験を実行する
run = experiment.submit(config=estimator)

# 実行中に実行の詳細を表示する
RunDetails(run).show()
run.wait_for_completion()

もう一度、完了した実行からメトリックと出力を取得できます。

In [ ]:
# 指標の記録を取得する
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

## モデルの新しいバージョンを登録する

新しいモデルをトレーニングしたので、ワークスペースに新しいバージョンとして登録できます。

In [ ]:
from azureml.core import Model

# モデルを登録する
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Parameterized SKLearn Estimator'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# 登録済みモデルを一覧表示する
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

探索が終了したら、この Notebook を閉じて、コンピューティング インスタンスをシャットダウンできます。